In [ ]:
import sqlite3
import os
import time
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
from IPython.display import clear_output as clear
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/嚮樂筆試')


command = ['/register', '/login', '/logout', '/listroom', '/entryroom', '/createroom', '/exitroom', '/say']


cur_user = ''
islogin = False
cur_roomidx = 0
list_message = []

def reset():
  global cur_user
  cur_user = ''
  global islogin
  islogin = False
  global cur_roomidx
  cur_roomidx = 0
  global list_message
  list_message = []

In [ ]:
def encrypt(text):
  tmp_str = ""
  for i in text:
    i = chr(ord(i) + 1)
    tmp_str +=i
  return tmp_str

def decrypt(text):
  tmp_str = ""
  for i in text:
    i = chr(ord(i) - 1)
    tmp_str +=i
  return tmp_str

def sendMessagetoDb(text):
  db_chatroom = sqlite3.connect('chatroom.db')
  cursor = db_chatroom.cursor()
  global cur_roomidx
  global cur_user
  tmp_str = encrypt(text)
  cursor.execute("INSERT INTO message_room"+str(cur_roomidx)+" ('Name', 'Message') VALUES('"+cur_user+"','"+tmp_str+"');")
  db_chatroom.commit()

  cursor.close()
  db_chatroom.close()


def refreshMessage():
  global cur_roomidx
  db_chatroom = sqlite3.connect('chatroom.db')
  cursor = db_chatroom.cursor()
  cursor.execute("SELECT * FROM 'message_room"+str(cur_roomidx)+"';")
  messages = cursor.fetchall()

  cursor.close()
  db_chatroom.close()

  global list_message
  list_message = []
  for m in messages:
    MessageId, Name, Message = m
    tmp_str = decrypt(Message)
    tmp = Name+" say: "+ tmp_str
    list_message.append(tmp)


def showMessage():
  output.clear_output()
  for m in list_message:
    print(m)

def exitRoom():
  global cur_roomidx
  cur_roomidx = 0

def sendMessage(text):
  global list_message
  global  cur_user
  list_message.append(cur_user+" say: "+ text)
  sendMessagetoDb(text)


In [ ]:
########################
##room
########################

def getRoomInfo():

  db_chatroom = sqlite3.connect('chatroom.db')
  cursor = db_chatroom.cursor()
  cursor.execute("SELECT * FROM 'room';")
  rooms = cursor.fetchall()
  room_list_button = []
  ##列出所有房間
  for r in rooms:
    room_id, room_name = r
    button_room = widgets.Button(description = "房間:"+room_name, layout={'width': '300px','color':'blue'})
    button_room.on_click(func_button_roominfo)
    room_list_button.append(button_room)

  output = widgets.Output()
  display(*room_list_button, output)
  cursor.close()
  db_chatroom.close()
###############################
def createRoom(_roomname):
  global cur_roomidx

  try:
    db_chatroom = sqlite3.connect('chatroom.db')
    cursor = db_chatroom.cursor()
    cursor.execute("SELECT * FROM 'room';")
    rooms = cursor.fetchall()

    iscreated = False
    for r in rooms:
      room_id, room_name = r
      if(room_name == _roomname):
        print("roomname existed")
        iscreated = True
        break

    if(iscreated == False):
      cursor.execute("INSERT INTO room(RoomName) VALUES('"+_roomname+"');")
      db_chatroom.commit()

      cursor.execute("SELECT * FROM 'room';")
      rooms = cursor.fetchall()
      for r in rooms:
        room_id, room_name = r
        if(room_name == _roomname):
          cur_roomidx = room_id
          break

      #創建table->存message
      cursor.execute("CREATE TABLE message_room"+str(cur_roomidx)+"(MessageId INTEGER NOT NULL, Name Text, Message Text, PRIMARY KEY(MessageId));")
      db_chatroom.commit()

    cursor.close()
    db_chatroom.close()

    time.sleep(2)
    clear()
    return (iscreated == False)

  except:
    print("error when open database")


def entryRoom(_roomname):
  db_chatroom = sqlite3.connect('chatroom.db')
  cursor = db_chatroom.cursor()
  cursor.execute("SELECT * FROM 'room';")
  rooms = cursor.fetchall()

  for r in rooms:
    room_id, room_name = r
    if(room_name == _roomname.split('房間:')[1]):
      global cur_roomidx
      cur_roomidx = room_id
      break

  cursor.close()
  db_chatroom.close()
  print("success entry "+room_name)
  time.sleep(2)
  clear()
  return True;




In [ ]:
########################
##user
########################

def userLogin(_username, _password):
  global cur_user
  global islogin

  db_chatroom = sqlite3.connect('chatroom.db')
  cursor = db_chatroom.cursor()
  cursor.execute("SELECT * FROM 'user';")
  users = cursor.fetchall()

  for r in users:
    id, name, password = r
    if(name == _username and password == _password):
      cur_user = name
      islogin = True
      break

  cursor.close()
  db_chatroom.close()


  if(islogin == True):

    print("success login "+cur_user)
    time.sleep(2)
    clear()
    return True;
  else:
    print("name or password error")
    time.sleep(2)
    output.clear_output()
    return False;


def userReg(_username, _password):
  global cur_user
  global islogin

  try:
    db_chatroom = sqlite3.connect('chatroom.db')
    cursor = db_chatroom.cursor()
    cursor.execute("SELECT * FROM 'user';")
    users = cursor.fetchall()

    isreg = False
    for r in users:
      id, name, password = r
      if(name == _username):
        print("username existed")
        isreg = True
        break

    if(isreg == False):
      cursor.execute("INSERT INTO user(Name, Password) VALUES('"+_username+"', '"+_password+"');")
      db_chatroom.commit()

      cur_user = _username
      islogin = True
      print("success register "+cur_user)

    cursor.close()
    db_chatroom.close()

    time.sleep(2)
    clear()
    return (isreg == False)

  except:
    print("error when open database")

def userLogout():
  reset()


In [ ]:
########################
##ui input block
########################

def userinput(name, password):
  return name, password
login_input = interactive(userinput, name=widgets.Combobox(), password=widgets.Combobox())

def roomnameinput(name):
  return name
roomname_input = interactive(roomnameinput, name=widgets.Combobox())

def messageinput(message):
  return message
message_input = interactive(messageinput, message=widgets.Combobox())


########################
##ui botton
########################
button_login = widgets.Button(description = "Login", layout={'border': '1px solid black'})
button_logout = widgets.Button(description = "Logout", layout={'border': '1px solid black'})
button_register = widgets.Button(description = "Register", layout={'border': '1px solid black'})
button_createroom = widgets.Button(description = "Create room", layout={'border': '1px solid black'})
button_exitroom = widgets.Button(description = "Exit room", layout={'border': '1px solid black'})
button_sendmessage = widgets.Button(description = "Send message", layout={'border': '1px solid black'})
output = widgets.Output()

def show_loginbotton():
  clear()
  global islogin
  global cur_roomidx
  if(islogin):
    display(button_logout)
    if(cur_roomidx == 0):
      display(button_createroom, output)
      display(roomname_input)
    else:
      display(button_exitroom, output)
      display(button_sendmessage, output)
      display(message_input)

  else:
    display(button_login, button_register, output)
    display(login_input)

########################
##callback func
########################

def func_login(b):
  input_name, input_password = login_input.result
  if(userLogin(input_name, input_password)):
    show_loginbotton()
    getRoomInfo()
  else:
    show_loginbotton()


def func_reg(b):
  input_name, input_password = login_input.result
  if(userReg(input_name, input_password)):
    show_loginbotton()
    getRoomInfo()
  else:
    show_loginbotton()


def func_logout(b):
  userLogout()
  show_loginbotton()

def func_createroom(b):
  input_name = roomname_input.result
  if(createRoom(input_name)):
    show_loginbotton()
    refreshMessage()
    showMessage()

  else:
    show_loginbotton()
    getRoomInfo()

def func_button_roominfo(b):
  if(entryRoom(b.description)):
    show_loginbotton()
    refreshMessage()
    showMessage()

  else:
    show_loginbotton()
    getRoomInfo()

def func_button_exitroom(b):
  exitRoom()
  show_loginbotton()
  getRoomInfo()

def func_button_sendmessage(b):
  input_message = message_input.result
  sendMessage(input_message)
  show_loginbotton()
  refreshMessage()
  showMessage()


button_login.on_click(func_login)
button_register.on_click(func_reg)
button_logout.on_click(func_logout)
button_createroom.on_click(func_createroom)
button_exitroom.on_click(func_button_exitroom)
button_sendmessage.on_click(func_button_sendmessage)

In [ ]:
output.clear_output()
reset()
show_loginbotton()



In [ ]:
output.clear_output()
clear()